In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import time
import pandas as pd
import torch.nn.functional as F
import torch.nn.utils.prune as prune



In [6]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load CIFAR-10 Dataset
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)



Using device: cpu
Files already downloaded and verified
Files already downloaded and verified


In [7]:
# Model definition (using the previously defined ComplexCNN)
class ComplexCNN(nn.Module):
    def __init__(self):
        super(ComplexCNN, self).__init__()

        # Conv Blocks
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Dummy FC layer (to be replaced)
        self.fc1 = None

        # Calculate FC input size dynamically
        self._initialize_fc()

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))

        # Flatten for FC layers
        x = x.view(x.size(0), -1)  # Flatten to [batch_size, features]
        x = self.fc1(x)
        return x

    def _initialize_fc(self):
        # Pass a dummy input to calculate FC input size
        dummy_input = torch.zeros(1, 3, 32, 32)  # CIFAR-10 input size
        x = self.pool1(F.relu(self.conv1(dummy_input)))
        x = self.pool2(F.relu(self.conv2(x)))
        fc_input_size = x.view(x.size(0), -1).size(1)
        self.fc1 = nn.Linear(fc_input_size, 10)




In [8]:
# Pruning Methods
def apply_structured_pruning(model, amount=0.5):
    """
    Applies structured pruning to the model by pruning entire filters.
    """
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d):
            prune.ln_structured(module, name='weight', amount=amount, n=2, dim=0)  # 수정된 부분

def apply_unstructured_pruning(model, amount=0.5):
    """
    Applies unstructured pruning to the model by pruning individual weights.
    """
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            torch.nn.utils.prune.random_unstructured(module, name='weight', amount=amount)



In [9]:
# Evaluation Function
def evaluate_model(model, data_loader):
    """
    Measures accuracy and inference time for a given model.
    """
    model.eval()
    correct = 0
    total = 0
    start_time = time.time()

    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    inference_time = time.time() - start_time
    accuracy = 100 * correct / total
    return inference_time, accuracy



In [10]:
# Training Function (to fine-tune pruned models)
def train_model(model, train_loader, criterion, optimizer, epochs=3):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

# Main Comparison
original_model = ComplexCNN().to(device)

# Pretrained model loading (if available)
# torch.save(original_model.state_dict(), "original_cnn.pt")
# original_model.load_state_dict(torch.load("original_cnn.pt"))

# Optimizer and loss
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(original_model.parameters(), lr=0.001)

# Train the original model (if not pre-trained)
train_model(original_model, train_loader, criterion, optimizer, epochs=3)

# Clone the model for pruning
structured_pruned_model = ComplexCNN().to(device)
unstructured_pruned_model = ComplexCNN().to(device)

structured_pruned_model.load_state_dict(original_model.state_dict())
unstructured_pruned_model.load_state_dict(original_model.state_dict())



<All keys matched successfully>

In [11]:
# Apply Pruning
apply_structured_pruning(structured_pruned_model, amount=0.5)
apply_unstructured_pruning(unstructured_pruned_model, amount=0.5)

# Fine-tune Pruned Models
structured_optimizer = optim.Adam(structured_pruned_model.parameters(), lr=0.001)
unstructured_optimizer = optim.Adam(unstructured_pruned_model.parameters(), lr=0.001)

train_model(structured_pruned_model, train_loader, criterion, structured_optimizer, epochs=3)
train_model(unstructured_pruned_model, train_loader, criterion, unstructured_optimizer, epochs=3)



In [12]:
# Evaluate Models
results = []
for i in range(5):  # Perform multiple evaluation iterations
    print(f"Iteration {i+1}/5")

    original_time, original_accuracy = evaluate_model(original_model, test_loader)
    structured_time, structured_accuracy = evaluate_model(structured_pruned_model, test_loader)
    unstructured_time, unstructured_accuracy = evaluate_model(unstructured_pruned_model, test_loader)

    results.append({
        "Iteration": i + 1,
        "Original_Inference_Time": original_time,
        "Original_Accuracy": original_accuracy,
        "Structured_Inference_Time": structured_time,
        "Structured_Accuracy": structured_accuracy,
        "Unstructured_Inference_Time": unstructured_time,
        "Unstructured_Accuracy": unstructured_accuracy,
    })



Iteration 1/5
Iteration 2/5
Iteration 3/5
Iteration 4/5
Iteration 5/5


In [13]:
# Convert results to DataFrame and compute statistics
df = pd.DataFrame(results)
df['Original_Time_Mean'] = df['Original_Inference_Time'].mean()
df['Original_Accuracy_Mean'] = df['Original_Accuracy'].mean()
df['Structured_Time_Mean'] = df['Structured_Inference_Time'].mean()
df['Structured_Accuracy_Mean'] = df['Structured_Accuracy'].mean()
df['Unstructured_Time_Mean'] = df['Unstructured_Inference_Time'].mean()
df['Unstructured_Accuracy_Mean'] = df['Unstructured_Accuracy'].mean()

df.to_csv("pruning_comparison.csv", index=False)
print("Results saved to pruning_comparison.csv")


Results saved to pruning_comparison.csv
